    1º mirar si un juego se ha vendido o no (hacer columna en tabala de juegos (1 vendido,0 no vendido))
    2º Agrupar por género los que se han vendido (contarlos) y dado su precio intentar predecir si se venderá
    otro tipo...
    3º Tener en cuenta el descuento aplicado en el caso de que hay tenido descuento (pocos casos)

In [123]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
 
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')
myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient['stum_for_you']
mycol = mydb["multidimensional"]

compra = mycol.find({"movimiento.tipo_movimiento":"compra"})
venta = mycol.find({"movimiento.tipo_movimiento":"venta"})
dfc = pd.DataFrame(list(compra))
dfv = pd.DataFrame(list(venta))
 

## Compras

### Corrección de celdas nulas por celdas con valores nulos

In [125]:
for col in dfc:
    for i, row_value in dfc[col].iteritems():
        if col == "dlc" and dfc[col].isna()[i]:
            dfc[col][i]={'puntero_dlc': -1, 'tipo': 'vacio', 'precio_dlc': -1}
        if col == "juego" and dfc[col].isna()[i]:
            dfc[col][i]={'puntero_juego': -1, 'genero': [], 'precio_juego': -1, 'clasificacion':'vacio'}

In [129]:
dfc

,_id,dlc,fecha,juego,movimiento,proveedor
0,5e2040ad37d0e4054c2cf0fe,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '03', 'dia': '04'}","{'puntero_juego': 8611, 'genero': ['action'], ...","{'precio_final': 4, 'tipo_movimiento': 'compra...","{'puntero_proveedor': 30, 'pais': 'Eritrea', '..."
1,5e2040ad37d0e4054c2cf0fa,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '12', 'dia': '31'}","{'puntero_juego': 7931, 'genero': ['strategy',...","{'precio_final': 20, 'tipo_movimiento': 'compr...","{'puntero_proveedor': 21, 'pais': 'Israel', 'v..."
2,5e2040ad37d0e4054c2cf0ff,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '07', 'dia': '07'}","{'puntero_juego': 3042, 'genero': ['action'], ...","{'precio_final': 16, 'tipo_movimiento': 'compr...","{'puntero_proveedor': 6, 'pais': 'Saint Helena..."
3,5e2040ad37d0e4054c2cf100,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '11', 'dia': '19'}","{'puntero_juego': 2073, 'genero': [], 'precio_...","{'precio_final': 80, 'tipo_movimiento': 'compr...","{'puntero_proveedor': 9, 'pais': 'Honduras', '..."
4,5e2040ad37d0e4054c2cf101,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '08', 'dia': '03'}","{'puntero_juego': 1239, 'genero': ['action'], ...","{'precio_final': 2, 'tipo_movimiento': 'compra...","{'puntero_proveedor': 13, 'pais': 'Poland', 'v..."
5,5e2040ad37d0e4054c2cf102,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '07', 'dia': '07'}","{'puntero_juego': 12673, 'genero': ['action', ...","{'precio_final': 0, 'tipo_movimiento': 'compra...","{'puntero_proveedor': 19, 'pais': 'Moldova', '..."
6,5e2040ad37d0e4054c2cf103,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '07', 'dia': '04'}","{'puntero_juego': 8854, 'genero': ['action', '...","{'precio_final': 4, 'tipo_movimiento': 'compra...","{'puntero_proveedor': 40, 'pais': 'Bermuda', '..."
7,5e2040ad37d0e4054c2cf104,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '12', 'dia': '27'}","{'puntero_juego': 6249, 'genero': ['adventure'...","{'precio_final': 8, 'tipo_movimiento': 'compra...","{'puntero_proveedor': 2, 'pais': 'Croatia', 'v..."
8,5e2040ad37d0e4054c2cf106,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '09', 'dia': '13'}","{'puntero_juego': 4667, 'genero': ['adventure'...","{'precio_final': 5, 'tipo_movimiento': 'compra...","{'puntero_proveedor': 38, 'pais': 'Benin', 'va..."
9,5e2040ad37d0e4054c2cf105,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '01', 'dia': '23'}","{'puntero_juego': 4683, 'genero': ['casual', '...","{'precio_final': 6, 'tipo_movimiento': 'compra...","{'puntero_proveedor': 32, 'pais': 'Angola', 'v..."


In [130]:
dfcdlc = pd.DataFrame(list(dfc.dlc))
dfcjuego = pd.DataFrame(list(dfc.juego))
dfcmovimiento = pd.DataFrame(list(dfc.movimiento))
dfcfecha = pd.DataFrame(list(dfc.fecha))
dfcproveedor = pd.DataFrame(list(dfc.proveedor))

## Ventas

In [128]:
for col in dfv:
    for i, row_value in dfv[col].iteritems():
        if col == "dlc" and dfv[col].isna()[i]:
            dfv[col][i]={'puntero_dlc': -1, 'tipo': 'vacio', 'precio_dlc': -1}
        if col == "juego" and dfv[col].isna()[i]:
            dfv[col][i]={'puntero_juego': -1, 'genero': [], 'precio_juego': -1, 'clasificacion':'vacio'}
dfv

,_id,descuento,dlc,fecha,juego,movimiento,ubicacion,usuario
0,5e20417437d0e442ac80d8dd,NaN,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '06', 'dia': '03'}","{'puntero_juego': 2117, 'genero': ['adventure'...","{'precio_final': 15, 'tipo_movimiento': 'venta...","{'pais': 'Spain', 'ciudad': 'Alacant'}","{'puntero_usuario': 1, 'sexo': 'H', 'fechaNaci..."
1,5e20417437d0e442ac80d8de,NaN,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '07', 'dia': '07'}","{'puntero_juego': 6063, 'genero': ['RPG'], 'pr...","{'precio_final': 3, 'tipo_movimiento': 'venta'...","{'pais': 'Spain', 'ciudad': 'Ceuta'}","{'puntero_usuario': 2, 'sexo': 'M', 'fechaNaci..."
2,5e20417437d0e442ac80d8df,NaN,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2017', 'mes': '11', 'dia': '08'}","{'puntero_juego': 3993, 'genero': ['RPG'], 'pr...","{'precio_final': 21, 'tipo_movimiento': 'venta...","{'pais': 'Spain', 'ciudad': 'Ceuta'}","{'puntero_usuario': 2, 'sexo': 'M', 'fechaNaci..."
3,5e20417437d0e442ac80d8e0,NaN,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '12', 'dia': '25'}","{'puntero_juego': 11392, 'genero': ['adventure...","{'precio_final': 15, 'tipo_movimiento': 'venta...","{'pais': 'Spain', 'ciudad': 'Ceuta'}","{'puntero_usuario': 2, 'sexo': 'M', 'fechaNaci..."
4,5e20417437d0e442ac80d8e2,NaN,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2017', 'mes': '05', 'dia': '17'}","{'puntero_juego': 5951, 'genero': ['action', '...","{'precio_final': 7, 'tipo_movimiento': 'venta'...","{'pais': 'Spain', 'ciudad': 'Palma de Mallorca'}","{'puntero_usuario': 4, 'sexo': 'H', 'fechaNaci..."
5,5e20417437d0e442ac80d8e1,NaN,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2017', 'mes': '10', 'dia': '22'}","{'puntero_juego': 10240, 'genero': ['action', ...","{'precio_final': 5, 'tipo_movimiento': 'venta'...","{'pais': 'Spain', 'ciudad': 'Santa Coloma de G...","{'puntero_usuario': 3, 'sexo': 'M', 'fechaNaci..."
6,5e20417437d0e442ac80d8e3,NaN,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '08', 'dia': '10'}","{'puntero_juego': 2699, 'genero': ['casual', '...","{'precio_final': 0, 'tipo_movimiento': 'venta'...","{'pais': 'Spain', 'ciudad': 'Palma de Mallorca'}","{'puntero_usuario': 4, 'sexo': 'H', 'fechaNaci..."
7,5e20417437d0e442ac80d8e4,NaN,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2020', 'mes': '01', 'dia': '11'}","{'puntero_juego': 6519, 'genero': [], 'precio'...","{'precio_final': 0, 'tipo_movimiento': 'venta'...","{'pais': 'Spain', 'ciudad': 'Palma de Mallorca'}","{'puntero_usuario': 4, 'sexo': 'H', 'fechaNaci..."
8,5e20417437d0e442ac80d8e5,NaN,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2017', 'mes': '08', 'dia': '14'}","{'puntero_juego': 12202, 'genero': ['action', ...","{'precio_final': 13, 'tipo_movimiento': 'venta...","{'pais': 'Spain', 'ciudad': 'Oviedo'}","{'puntero_usuario': 5, 'sexo': 'H', 'fechaNaci..."
9,5e20417437d0e442ac80d8e6,NaN,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2017', 'mes': '03', 'dia': '06'}","{'puntero_juego': 1498, 'genero': ['casual'], ...","{'precio_final': 25, 'tipo_movimiento': 'venta...","{'pais': 'Spain', 'ciudad': 'Logroño'}","{'puntero_usuario': 6, 'sexo': 'H', 'fechaNaci..."


In [ ]:
dfvdlc = pd.DataFrame(list(dfc.dlc))
dfvjuego = pd.DataFrame(list(dfc.juego))
dfvmovimiento = pd.DataFrame(list(dfc.movimiento))
dfvfecha = pd.DataFrame(list(dfc.fecha))
dfvusuario = pd.DataFrame(list(dfc.usuario))
dfvdescuento = pd.DataFrame(list(dfc.descuento))
dfvubicacion = pd.DataFrame(list(dfc.ubicacion))

In [ ]:
##Hacer dataframes con los siguientes caracteristicas
#Compras de juegos sin descuentos
#Compras de DLCs sin descuentos
#Compras de juegos con descuentos
#Compras de DLCs con descuentos
#Ventas de juegos sin descuentos
#Ventas de DLCs sin descuentos
#Ventas de juegos con descuentos
#Ventas de DLCs con descuentos


#isnan para comporbar que es NaN